# The Quantum Chai Maker

Hey there! You're likely here because you love tea as much as I do. But did you know there's a whole world of flavors to explore? Let's use quantum computing to spice up your tea recipes, pun intended! 

Quantum computing is great for this flavor journey because it's probability based: every possible outcome (read, ingredient) has some probability of being selected!

Here are the ingredients the Quantum Chai Maker will choose from:


|  Tea | Mild Spices  | Zesty Spices  | Herbs  | Misc. Flavors  |
|---|---|---|---|---|
|  black |  cardamom (elaichi) | ginger (adrak)  |  peppermint |  orange peel |
|  green |  fennel seeds (saunf) |  black peppercorns | lemongrass  | rose petals  |
|   |  star anise |  white peppercorns | licorice root  |  lemon |
|   | bay leaves  |  cumin seeds (zeera) |  chamomile | almond shavings/extract  |
|   | cloves (laung)  | allspice  |  spearmint  |  cocoa |
|   | cinnamon  |  carom seeds (ajwain) |  coriander |   |
|   | nutmeg  | tumeric (haldi)  |   |   |
|   | saffron (kesar)  |   |   |   |
|   |  vanilla bean/extract |   |   |   |
|   | holy basil (tulsi)  |   |   |   |


There are two types of tea to choose from, and four flavor categories. Some Hindi translations are provided in parentheses.

### What the QCM does:

1. Chooses between green and black tea.
2. Selects ingredient categories. Any number of categories and category combinations can be selected, from just one to all four.
3. Chooses the number of ingredients it wants from each of its selected categories.
4. Provides a special list of ingredients!

Some points to note before you dive in:

- **Sweeteners:** Sweeteners haven't been added, I've left that up to you. But here are some options: white sugar, brown sugar, agave syrup, honey, maple syrup, cane sugar, and stevia.
- **Milk:** Milk hasn't been added either. I recommend milk with black tea if you're looking for something more along the lines of desi chai. Some people who brew desi chai enjoy 'cooking' the milk and tea together if they're using dairy milk. If you're using other types of milk (soya, almond, etc.) I suggest you add the milk after brewing the tea. 
- **Zesty spices:** Generally all flavorings should be used in small amounts so that none become overpowering, but take extra care with the spices in the 'Zesty Spices' category. 
- **Qiskit:** We use IBM's Qiskit for the quantum computing parts. If you haven't already, you'll need to install it: https://qiskit.org/documentation/install.html.

If you're curious about the quantum concepts used to build this, I've added splashes of information in blocks and comments. If you're just here to make some tea, simply `shift+enter` your way to the end!

**Disclaimer: Some combinations may turn out to be less than appealing. After all, we're working with probabilities, eh? You can choose which ingredients to use once the Chai Maker gives you a list! So, let's just agree to not blame me for any, erm, unpleasantness, shall we?**

### The Code 

First, we have to import all the important stuff. We use IBM's Qiskit for the quantum computing parts. All circuits will be simulated with their `qasm_simulator` but feel free to modify the code to run it on a real backend!

In [217]:
from qiskit import QuantumCircuit
import math
import numpy as np
import random
import itertools

Let's store the ingredients in neat little lists.

In [218]:
teas = ['black', 'green']

mild_spices = ['cardamom (elaichi)', 'fennel seeds (saunf)', 'star anise',  'bay leaves', 'cloves (laung)', 'cinnamon', 'nutmeg', 'saffron (kesar)', 'vanilla bean/extract', 'holy basil (tulsi)']

zesty_spices = [ 'ginger (adrak)', 'black peppercorns', 'white peppercorns',  'cumin seeds (zeera)',  'allspice', 'carom seeds (ajwain)', 'tumeric (haldi)']

herbs = ['peppermint', 'lemongrass', 'licorice root', 'chamomile', 'spearmint', 'coriander']

misc_flavors = ['orange peel', 'rose petals', 'lemon', 'almond shavings/extract', 'cocoa']



In order to choose between green and black tea, we make use of one of the four Bell states, which are really nifty maximally entangled two qubit states used in some pretty cool applications!

Specifically, we use the state:

$\mathcal{B} = \frac{1}{\sqrt{2}} (|01\rangle + |10\rangle)$

The equation shows a superposition of two states: $|01\rangle$ and $|10\rangle$, where the first position within the $|\rangle$ corresponds to the first qubit, and the second position corresponds to the second qubit.

This means the two qubits are entangled such that if the first qubit is measured to be 0, the second qubit will definitely be a 1. And vice versa.

In the context of tea, we're basically saying:

$|$not_green, black$\rangle + |$green, not_black$\rangle$

A measurement will result in just one of these two possibilities.

In [219]:
def which_tea():
    # creates the Bell state (|01> + |10>)/sqrt(2) to choose between black and green tea
    circ = QuantumCircuit(2,2) # create a quantum circuit with 2 qubits and 2 classical bits
    circ.h(0) # Hadamard gate on qubit 0
    circ.cx(0,1) # controlled x with qubit 0 as control and qubit 1 as target
    circ.x(0) # x on qubit 0
    circ.measure([0,1],[0,1]) # each qubit is measured, and the total outcome is either 01 or 10
    job = qiskit.execute(circ, qiskit.BasicAer.get_backend('qasm_simulator') ) # run on qasm_simulator
    result = job.result().get_counts() # result is a dict, with key = classical bit outcomes, value = number of counts
    max_res = max(result, key=result.get) # find the result with the highest count
    return teas[int(max_res,2)-1] #convert to decimal

For the rest of the decisions, we will simply make use of superposition. When a Hadamard gate operates on a qubit in the $|0\rangle$ state we get:

$H|0\rangle = \frac{1}{\sqrt{2}} (|0\rangle + |1\rangle)$

This means that upon measurement, the outcome is equally likely to be a 0 as it is a 1. There are two possible outcomes.

If we have multiple qubits, we can have even more possible outcomes! Specifically, if we have $n$ qubits, there are $2^n$ possible outcomes.

Suppose we want to choose a number between 0 and 7, that is, we have a choice among 8 numbers. If we use 3 qubits that are each in the above superposed state, then we have 8 possibilities at our disposal: 000, 001, 010, 011, 100, 101, 110, 111. Upon measurement, we get one of them. And if we convert them from binary to decimal, we get exatcly what we are looking for: a number between 0 and 7!

In [220]:
def run_QuantumCirc(n): # creates and runs a quantum circuit with a Hadamard operating on each qubit
    qr = qiskit.QuantumRegister(n) # create quantum register with n qubits
    cr = qiskit.ClassicalRegister(n) # create classical register with n bits
    circ  = qiskit.QuantumCircuit(qr, cr) # create circuit with the two registers
    circ.h(qr) # perform Hadamard  on each qubit
    circ.measure(qr,cr) # each qubit is measured, and the outcome for one qubit is either 0 or 1
    job = qiskit.execute(circ, qiskit.BasicAer.get_backend('qasm_simulator') ) 
    result = job.result().get_counts() 
    return result

And now comes the ingredient selection! 

In [223]:
def select_ingredients(category): # runs a quantum circuit to select the number of ingredients in a category
    num_choices = len(category)
    if math.log(num_choices,2)%int(math.log(num_choices,2)) == 0: # checks whether log(num_choices) to the base 2 is a whole number
        n = int(math.log(num_choices,2)) # n = number of qubits
        result = run_QuantumCirc(n)
        max_res = max(result, key=result.get) 
    else: 
        n = int(math.log(num_choices,2))+1 # adds 1 to log(N) to the base 2 to get total number of qubits needed
        result = run_QuantumCirc(n)
        max_res = max(result, key=result.get)
        while(int(max_res,2) > num_choices-1): # find max that is less than num_choices
            result.pop(str(max_res))
            max_res = max(result, key=result.get)
    selections = []
    random.shuffle(category) # randomly shuffles the category list
    for i in range(int(max_res,2)+1): # int(max_res,2)+1 is in decimal; it's the number of ingredients in the category that you will be using 
        selections.append(category[i]) # returns the first int(max_res,2)+1 entries in the shuffled list
    return selections

def choose_categories(tea): # chooses which ingredient categories your ingredients will come from, we use simple qubit superposition state |0>+|1>
    n = 4 #There are 4 ingredient categories
    categories_dict = {'0': select_ingredients(mild_spices), '1': select_ingredients(herbs), '2': select_ingredients(misc_flavors), '3': select_ingredients(zesty_spices)}
    max_res = '0000'
    final_ingredients = []
    while len(final_ingredients) == 0: # to make sure we don't return an empty list
        while max_res == '0000': # run it until we get SOME ingredients! No one wants 0 ingredients. That'd be utterly boring.
            result = run_QuantumCirc(n)
            max_res = max(result, key=result.get)  
        for ind in range(n):
            if max_res[ind] == '1':
                final_ingredients.append(categories_dict[str(ind)])
    
        final_ingredients = list(itertools.chain.from_iterable(final_ingredients))
        if tea == 'green' and 'cocoa' in final_ingredients: # cocoa doesn't really gel with green tea, in my opinion
            final_ingredients.remove('cocoa')
        if tea == 'black' and 'chamomile' in final_ingredients: # i personally feel chamomile doesn't really gel with black tea 
            final_ingredients.remove('chamomile')

    return final_ingredients

In [228]:
tea = which_tea()
print(f"""
Your quantum chai is {tea} tea with the following ingredients:
{choose_categories(tea)}

Happy drinking!""")




Your quantum chai is green tea with the following ingredients:
['cinnamon', 'cloves (laung)', 'nutmeg', 'holy basil (tulsi)', 'tumeric (haldi)', 'ginger (adrak)', 'cumin seeds (zeera)', 'white peppercorns', 'carom seeds (ajwain)', 'black peppercorns']

Happy drinking!
